Q2) 1) 2)

* Below are the functions to sample text from the model using different temperatures (i.e., 1/α values) and a function that uses an RNN to complete a string. Comments have been mentioned in the code itself.


* For low values of alpha, the probability vector covers more of the given vocabulary and is more spread out. This ensures that each character has a high probability of being sampled.


* For high values of alpha, the softmax function behaves like a max function. This ensures that the character with a higher probability gets picked. There would be repetition of the same word in the sentence.


* The RNN generated text has been produced that is a plausible continuation of a given starter string. In order to do that, I have computed the hidden activity h at the end of the starter string, and then to start generating new text. Included 5 interesting examples of outputs that my network generated using a starter string.


In [1]:
import numpy

data = open('shakespeare_train.txt', 'r').read()

sequence_length = 1500
hidden_size = 250

In [2]:
# Using the trained weights 
a = numpy.load(open("char-rnn-snapshot.npz"))
Wxh = a["Wxh"] 
Whh = a["Whh"] 
Why = a["Why"] 
bh = a["bh"] 
by = a["by"]
chars = a["chars"].tolist()
data_size = a["data_size"].tolist() 
vocab_size = a["vocab_size"].tolist()
char_to_ix = a["char_to_ix"].tolist() 
ix_to_char = a["ix_to_char"].tolist()

In [3]:
# Generating a sample text with an alpha value for temperature.
def temp(length, alpha=1):

  inputs = [char_to_ix[ch] for ch in data[:sequence_length]]
  hs = numpy.zeros((hidden_size,1))
  
  # generates a sample
  sample_ix = sample(hs, inputs[0], length, alpha)
  txt = ''.join(ix_to_char[ix] for ix in sample_ix)
  print '_____\n%s \n_______' % (txt, )       

In [5]:
#sample a sequence of integers from the model.
def sample(h, seed_ix, n, alpha):

  x = numpy.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ix_list = []
  for t in xrange(n):
    h = numpy.tanh(numpy.dot(Wxh, x) + numpy.dot(Whh, h) + bh)
    y = numpy.dot(Why, h) + by
    p = numpy.exp(alpha * y) / numpy.sum(numpy.exp(alpha * y))
    ix = numpy.random.choice(range(vocab_size), p=p.ravel())
    x = numpy.zeros((vocab_size, 1))
    x[ix] = 1
    ix_list.append(ix)
  return ix_list

In [9]:
# Complete a String
def comp(m, n):

  numpy.random.seed
  begin_loc = numpy.random.randint(260000) 
  inputs = [char_to_ix[ch] for ch in data[begin_loc : begin_loc+sequence_length]]
  h = numpy.zeros((hidden_size,1))
  x = numpy.zeros((vocab_size, 1))
  word_index = 0
  ix = inputs[word_index]
  x[ix] = 1

  ix_list = []
  ix_list.append(ix)
  for t in xrange(m):
    h = numpy.tanh(numpy.dot(Wxh, x) + numpy.dot(Whh, h) + bh)
    x = numpy.zeros((vocab_size, 1))
    ix = inputs[word_index + 1]
    word_index += 1
    x[ix] = 1
    ix_list.append(ix)
    # generates the context text
  txt = ''.join(ix_to_char[ix] for ix in ix_list)
  print 'Context text generated: \n___________\n %s \n________ \n' % (txt, )
    # use the output as the next input where we start the continuation and compute the probability
  y = numpy.dot(Why, h) + by
  p = numpy.exp(y) / numpy.sum(numpy.exp(y))
  ix = numpy.random.choice(range(vocab_size), p=p.ravel())
  x = numpy.zeros((vocab_size, 1))
  x[ix] = 1

  # complete the string
  ix_list = []
  for t in xrange(n):
    h = numpy.tanh(numpy.dot(Wxh, x) + numpy.dot(Whh, h) + bh)
    y = numpy.dot(Why, h) + by
    p = numpy.exp(y) / numpy.sum(numpy.exp(y))
    ix = numpy.random.choice(range(vocab_size), p=p.ravel())
    x = numpy.zeros((vocab_size, 1))
    x[ix] = 1
    ix_list.append(ix)
  
  # Generate the continuation
  txt = ''.join(ix_to_char[ix] for ix in ix_list)
  print 'Continuation text generated: \n_________\n %s \n__________\n' % (txt, )

In [12]:
temp(100,alpha=5)

_____
irst Senath so the the the the the the the the the counter the the and the come the and the the the  
_______


In [13]:
temp(100,alpha=1)

_____
irst Cyely:
I'l con'touve with arcould comand. Theike a were the to wit man.

BRUTUS:
It shingal lac 
_______


In [14]:
temp(100,alpha=0.1)

_____
ysW'Wbnrp,??
!S-kNqYgC.m--wcU!hfni?nr-wEOZDd
HPy,
'xapjNIZAVKxy.UWBR;nPyEMpv'AKNiffs''V,EOQhAsomBui' 
_______


In [15]:
comp(100,200)

Context text generated: 
___________
 o, you that banish'd him;
A mile before his tent fall down, and knee
The way into his mercy: nay, if  
________ 

Continuation text generated: 
_________
 brepe thy mo, my bating no that,! for con you more be's not entary; for of bo?

SICINIUS:
Yim ok; onite with wise both she nervender comer.

SICINI:
Nad:
surseet canp. Aghen in ed strotitwas. whet fel 
__________



In [16]:
comp(300,200)

Context text generated: 
___________
 ius,
That they of Rome are entered in our counsels
And know how we proceed.

AUFIDIUS:
Is it not yours?
What ever have been thought on in this state,
That could be brought to bodily act ere Rome
Had circumvention? 'Tis not four days gone
Since I heard thence; these are the words: I think
I have the l 
________ 

Continuation text generated: 
_________
 ou come, ant takes; unour. O in hest me Pounthest, 'Tis
Let goss gromath de tay.

VOLUMNIA:
Seeltsthare?
O netbyce
To do trame as region him say knoth your of he stroine ny
cuest

VENBR:
No in mitizen 
__________



In [17]:
comp(500,700)

Context text generated: 
___________
 y happy days before thy death;
And, after many lengthen'd hours of grief,
Die neither mother, wife, nor England's queen!
Rivers and Dorset, you were standers by,
And so wast thou, Lord Hastings, when my son
Was stabb'd with bloody daggers: God, I pray him,
That none of you may live your natural age,
But by some unlook'd accident cut off!

GLOUCESTER:
Have done thy charm, thou hateful wither'd hag!

QUEEN MARGARET:
And leave out thee? stay, dog, for thou shalt hear me.
If heaven have any grievous  
________ 

Continuation text generated: 
_________
 

First I'll love shall'd of noble wopts
And had his art, he's pees, thou depo, I cond hearing gus thing hear gravered.! rot forlain a he'll
sed beher?

BRUTUS:
Wheld your me what
Rees-
The of clam, good
Whe will whigle onaing'd yes that the whanter voughery :
Dly sil woul' he rewemeem this enem not?
Of go to I'll as 'ttre oft go! my tere as all'lf,
Me.

CORIOLANUS:
Con cuves than, me neich hance
Thou so

In [18]:
comp(700,100)

Context text generated: 
___________
 ws have made me stay, I fled from words.
You soothed not, therefore hurt not: but
your people,
I love them as they weigh.

MENENIUS:
Pray now, sit down.

CORIOLANUS:
I had rather have one scratch my head i' the sun
When the alarum were struck than idly sit
To hear my nothings monster'd.

MENENIUS:
Masters of the people,
Your multiplying spawn how can he flatter--
That's thousand to one good one--when you now see
He had rather venture all his limbs for honour
Than one on's ears to hear it? Proceed, Cominius.

COMINIUS:
I shall lack voice: the deeds of Coriolanus
Should not be utter'd feebly. It is held
That valour is the chiefest virtue, and
Most dignifies the haver: if it be,
The man I speak  
________ 

Continuation text generated: 
_________
 in the praity my pee's weind have while a no thou soldy:
And that for you bood blind, the seromer:
A 
__________



In [19]:
comp(250,251)

Context text generated: 
___________
 gs, then cursed she Buckingham,
Then cursed she Richard. O, remember, God
To hear her prayers for them, as now for us
And for my sister and her princely sons,
Be satisfied, dear God, with our true blood,
Which, as thou know'st, unjustly must be spilt. 
________ 

Continuation text generated: 
_________
 be stwere hawny:
I king word I kinouesatinst,
And didy who as cowem!

MENENIUS:
I had mombe be that a caees pawnicl.

VOLUMNIA:
Whighter wot 'Tis, ind with; hid mee Halty:
Nor is axanainst day; I mee,
I'll finger lith groce:
hear,
A just fain thre our 
__________



Q2) 3) 4)

In the samples that the RNN generated, it seems that a newline or a space usually follows
the colon (i.e., :) character.

Wxh[100][9], Why[0][100] and Why[2][100] are the most responsible weights for
causing this type of behaviour.

When the character ":" is passed in as the one-of-K encoding where the 9th
entry in the input x is 1, the 9th column of Wxh gets picked up (Wxh[:, 9]).
Also, the 100th entry of the hidden units h is the most
activated with value of 0.9999, which is very large. Note that h being a tanh
unit asymptotically approaches 1. This makes Whh and the previous hidden
state insignificant. Wxh[100][9] is the most responsible for h[100] being very
large.

At the output stage, the hidden units is multiplied with the matrix Why.
The result is that y[0] (newline character) and y[2] (space character) take on
big values. This is because Why[0][100] and Why[2][100] are the
biggest values in the Why[:, 100] column when the hidden units are activated at
the 100th position.

Large y[0] and y[2] make the newline character and space character very
probable to be sampled after the softmax function turns the output vectors into
a probability vector. That is why the newline and space character always follow the colon character.